# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kruisfontein,-34.0033,24.7314,289.47,46,0,3.47,ZA,1696302180
1,1,kailua-kona,19.6406,-155.9956,301.21,78,86,1.34,US,1696302091
2,2,adamstown,-25.0660,-130.1015,293.51,74,38,10.01,PN,1696302121
3,3,port-aux-francais,-49.3500,70.2167,275.56,81,69,15.02,TF,1696302118
4,4,longyearbyen,78.2186,15.6401,270.06,68,75,8.23,SJ,1696302132


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo=True,
    tiles='OSM',
    frame_width=700,
    frame_height=500,
    size='Humidity',
    scale=0.50,
    color="blue",  
    title='City Data: Humidity Visualization'
)

# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities = city_data_df[(city_data_df['Humidity'] < 40) & (city_data_df['Max Temp'] > (303))]

# Drop any rows with null values
ideal_cities = ideal_cities.dropna()

# Display sample data
print(ideal_cities.head())

     City_ID            City      Lat       Lng  Max Temp  Humidity  \
108      108       mount isa -20.7333  139.5000    307.02        17   
250      250           dubbo -32.2500  148.6167    303.22        21   
291      291  porto murtinho -21.6989  -57.8825    305.22        35   
311      311        tidjikja  18.5564  -11.4271    303.99        16   
321      321        karratha -20.7377  116.8463    305.16        22   

     Cloudiness  Wind Speed Country        Date  
108           1        6.69      AU  1696302072  
250          62       11.32      AU  1696302172  
291         100        1.89      BR  1696302129  
311          73        2.64      MR  1696302202  
321          54        6.24      AU  1696301898  


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
108,mount isa,AU,-20.7333,139.5000,17,
250,dubbo,AU,-32.2500,148.6167,21,
291,porto murtinho,BR,-21.6989,-57.8825,35,
311,tidjikja,MR,18.5564,-11.4271,16,
321,karratha,AU,-20.7377,116.8463,22,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "limit":1,
    "radius":radius,
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mount isa - nearest hotel: Ibis Styles
dubbo - nearest hotel: Blue Gum Motel
porto murtinho - nearest hotel: Hotel Saladero Cuê
tidjikja - nearest hotel: No hotel found
karratha - nearest hotel: Karratha International Hotel
akjoujt - nearest hotel: فندق فاطمة
zouerate - nearest hotel: فندق تيرس زمور
sanctuary point - nearest hotel: Dungowan Accommodation
alice springs - nearest hotel: Aurora Alice Springs
conceicao do araguaia - nearest hotel: No hotel found
goundam - nearest hotel: No hotel found
howard springs - nearest hotel: No hotel found
turbat - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
108,mount isa,AU,-20.7333,139.5000,17,Ibis Styles
250,dubbo,AU,-32.2500,148.6167,21,Blue Gum Motel
291,porto murtinho,BR,-21.6989,-57.8825,35,Hotel Saladero Cuê
311,tidjikja,MR,18.5564,-11.4271,16,No hotel found
321,karratha,AU,-20.7377,116.8463,22,Karratha International Hotel
348,akjoujt,MR,19.6969,-14.4164,13,فندق فاطمة
370,zouerate,MR,22.7187,-12.4521,15,فندق تيرس زمور
371,sanctuary point,AU,-35.1114,150.6290,33,Dungowan Accommodation
470,alice springs,AU,-23.7000,133.8833,14,Aurora Alice Springs
512,conceicao do araguaia,BR,-8.2578,-49.2647,36,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(x='Lng',
                            y='Lat',
                            geo=True,
                            tiles='OSM',
                            frame_width = 700,
                            frame_height = 500,
                            size = 'Humidity',
                            scale = 1.0,
                            color = "blue",
                            hover_cols = ['Hotel Name'])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name)